In [ ]:
# Code to create such files and plots to demonstrate the most popular check-in time during the week for\
# individual cities
import get_popular_time
# create a bunch of files that record the time adn average check-ins time for each city, but we only record\
# 10 cities with the most number of check-ins
get_popular_time.create_popular_time_cities()
get_popular_time.plot_popular_time_all_cities()
